<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#test" data-toc-modified-id="test-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>test</a></span></li></ul></div>

In [1]:
import pandas as pd
import copy
import pprint
import xlsxwriter
#显示所有列
pd.set_option('display.max_columns', 20)
# pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 20)
# pd.set_option('display.max_rows', None)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)

In [2]:
class My_DataFrame(pd.DataFrame):
    def __init__(self, data=None, index=None, columns=None, dtype=None, copy=False):
        pd.DataFrame.__init__(self, data, index, columns, dtype, copy)
 
    def my_mergewr_excel(self,path,key_cols=None,merge_cols=[]):
        # sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True):
        self_copy=My_DataFrame(self,copy=True)
        line_cn=self_copy.index.size
        cols=list(self_copy.columns.values)
#         if all([v in cols for i,v in enumerate(key_cols)])==False:     #校验key_cols中各元素 是否都包含与对象的列
#             print("key_cols is not completely include object's columns")
#             return False
        if all([v in cols for i,v in enumerate(merge_cols)])==False:  #校验merge_cols中各元素 是否都包含与对象的列
            print("merge_cols is not completely include object's columns")
            return False    
 
        wb2007 = xlsxwriter.Workbook(path)
        worksheet2007 = wb2007.add_worksheet()
        format_top = wb2007.add_format({'border':1,'bold':True,'text_wrap':True})
        format_other = wb2007.add_format({'border':1,'valign':'vcenter'})
        for i,value in enumerate(cols):  #写表头
            #print(value)
            worksheet2007.write(0,i,value,format_top)
        
        #merge_cols=['B','A','C']
        #key_cols=['A','B']
        if key_cols == None:   #如果key_cols 参数不传值，则无需合并
            self_copy['RN']=1
            self_copy['CN']=1
        else:
            self_copy['RN']=0
            self_copy['CN']=0
            start_pos=0
            end_pos=0
            pre_key =self_copy.loc[0,key_cols]
        
        # generate RN and CN
            for i in range(line_cn):
                if self_copy.loc[i,key_cols] == pre_key:
                    end_pos = i
#                     print(i)
                else:
                    gap = end_pos-start_pos+1
                    for j in range(start_pos, end_pos+1):
                        self_copy.loc[j,'CN'] = gap
                        self_copy.loc[j,'RN'] = (j-start_pos)+1
#                     print("start_pos:"+str(start_pos)+",end_pos:"+str(end_pos)+',gap:'+str(end_pos-start_pos+1))
#                     print(pre_key)
                    start_pos = i
                    end_pos = start_pos
#                     print(i)
                    pre_key = self_copy.loc[i,key_cols]
            end_pos = line_cn-1
            gap = end_pos-start_pos+1
            for j in range(start_pos, end_pos+1):
                self_copy.loc[j,'CN'] = gap
                self_copy.loc[j,'RN'] = (j-start_pos)+1            
#             print("start_pos:"+str(start_pos)+",end_pos:"+str(line_cn-1)+',gap:'+str(line_cn-1-start_pos+1))
#             print(pre_key)
            
            
#             self_copy['RN']=self_copy.groupby(key_cols,as_index=False).rank(method='first').iloc[:,0] #以key_cols作为是否合并的依据
#             self_copy['CN']=self_copy.groupby(key_cols,as_index=False).rank(method='max').iloc[:,0]
        #print(self)
        for i in range(line_cn):
            if self_copy.loc[i,'CN']>1:
                #print('该行有需要合并的单元格')
                for j,col in enumerate(cols):
                    #print(self_copy.loc[i,col])
                    if col in (merge_cols):   #哪些列需要合并
                        if self_copy.loc[i,'RN']==1:  #合并写第一个单元格，下一个第一个将不再写
                            worksheet2007.merge_range(i+1,j,i+int(self_copy.loc[i,'CN']),j, self_copy.loc[i,col],format_other) ##合并单元格，根据LINE_SET[7]判断需要合并几个
                            #worksheet2007.write(i+1,j,df.loc[i,col])
                        else:
                            pass
                        #worksheet2007.write(i+1,j,df.loc[i,j])
                    else:
                        worksheet2007.write(i+1,j,self_copy.loc[i,col],format_other)
                    #print(',')
            else:
                #print('该行无需要合并的单元格')
                for j,col in enumerate(cols):
                    #print(df.loc[i,col])
                    worksheet2007.write(i+1,j,self_copy.loc[i,col],format_other)
                
                
        wb2007.close()
        self_copy.drop('CN', axis=1)
        self_copy.drop('RN', axis=1)
        
 

In [4]:
def show_transform_process(vl_spec,data,label_attr):
    filters = []
    agg_dict = None
    fields = set()
    fields.add(label_attr)
    for fil in vl_spec['transform']:
        if type(fil['filter']) == type({}):
            tp = 'dict'
            field = fil['filter']['field']
            range_ = fil['filter']['oneOf']
            record = {'type':'filter_dict', 'field':field, 'range':range_}
        elif type(fil['filter']) == type(''):
            tp = 'str'
            left_idx_field = fil['filter'].find('["')+2
            right_idx_field = fil['filter'].find('"]')
            field = fil['filter'][left_idx_field:right_idx_field]
            left_idx_symbol = fil['filter'].find(']) ')+3
            right_idx_symbol = fil['filter'].find(']) ')+4  
            symbol = fil['filter'][left_idx_symbol:right_idx_symbol]
            left_idx_value = fil['filter'].find(']) ')+5
            value = float(fil['filter'][left_idx_value:])
            record = {'type':'filter_str', 'field':field, 'symbol':symbol,"value":value}
        fields.add(field)
        filters.append(record)
    for chan in vl_spec['encoding']:
        fields.add(vl_spec['encoding'][chan]['field'])
    agg_x = False
    agg_y = False    
    if 'aggregate' in vl_spec['encoding']['x']:
        agg_x = True
        agg_dict = {'type': 'aggregation', 'op':vl_spec['encoding']['x']['aggregate'], 'field':vl_spec['encoding']['x']['field'], 'groupby':vl_spec['encoding']['y']['field']}
    if 'aggregate' in vl_spec['encoding']['y']:
        agg_y = True
        agg_dict = {'type': 'aggregation', 'op':vl_spec['encoding']['y']['aggregate'], 'field':vl_spec['encoding']['y']['field'], 'groupby':vl_spec['encoding']['x']['field']}
    if (agg_x and agg_y):
        raise("bad agg")
        return
#     print(filters)
#     print(agg_dict)
#     print(fields)
    df = pd.read_csv(data) 
    fields = list(fields)
    # generate overview
    overview_df = df[fields]
    print('------Overview of all the relevant fields-------')
    display(overview_df)
    # generate intermediate result of filter 
    filter_ct = 0
    pre_df = overview_df
    for fil in filters:
        filter_ct = filter_ct+1
        if fil['type'] == 'filter_str':
            if fil['symbol'] == '>':
                pre_df=pre_df[pre_df[fil['field']] > fil['value']]
            elif fil['symbol'] == '<':
                pre_df=pre_df[pre_df[fil['field']] < fil['value']]
            else:
                raise("new symbol unexpected")
        elif fil['type'] == 'filter_dict':
            subsets=[]
            for value in fil['range']:
                subsetCur_filter_df = pre_df[pre_df[fil['field']] == value]
                subsets.append(subsetCur_filter_df)
            pre_df =  pd.concat(subsets)
        print('------After filter No.'+str(filter_ct)+'-------')
        print(fil)
        display(pre_df)
    #generate intermediate result of aggregation 
    if agg_dict is not None:
        if agg_dict['op'] == 'mean':
            ref_dict=pre_df.groupby([agg_dict['groupby']]).mean([agg_dict['field']]).to_dict()
            group__dict = pre_df.groupby([agg_dict['groupby'],label_attr]).mean([agg_dict['field']]).to_dict()
        elif agg_dict['op'] == 'sum':
            ref_dict=pre_df.groupby([agg_dict['groupby']]).sum([agg_dict['field']]).to_dict()
            group__dict = pre_df.groupby([agg_dict['groupby'],label_attr]).sum([agg_dict['field']]).to_dict()
        elif agg_dict['op'] == 'min':
            ref_dict=pre_df.groupby([agg_dict['groupby']]).min([agg_dict['field']]).to_dict()
            group__dict = pre_df.groupby([agg_dict['groupby'],label_attr]).min([agg_dict['field']]).to_dict()
        elif agg_dict['op'] == 'max':
            ref_dict=pre_df.groupby([agg_dict['groupby']]).max([agg_dict['field']]).to_dict()
            group__dict = pre_df.groupby([agg_dict['groupby'],label_attr]).max([agg_dict['field']]).to_dict()
        new_group_dict=copy.deepcopy(group__dict)
        column_name_of_agg = agg_dict['op']+' '+agg_dict['field']
        new_group_dict[column_name_of_agg]={}
        for item in group__dict[agg_dict['field']]:
            new_group_dict[column_name_of_agg][item]=ref_dict[agg_dict['field']][item[0]]
        output_df=pd.DataFrame(new_group_dict)
        output_df.index.names=[agg_dict['groupby'],label_attr]
        final=output_df.reset_index()
        final.to_csv('final.csv',index=False)
        print('------After Aggregation'+str(filter_ct)+'-------')
        print(agg_dict)
        display(final)    
        output_df = My_DataFrame(pd.read_csv('final.csv'))
        output_df.my_mergewr_excel('final_merged.xlsx',agg_dict['groupby'],[agg_dict['groupby'],column_name_of_agg])
    return

# test

In [5]:
data_position = '../../assets/data/movies-w-year.csv'
vl_spec = {'$schema': 'https://vega.github.io/schema/vega-lite/v4.json',
 'data': {'format': {'type': 'csv'},
          'url': '../../assets/data/movies-w-year.csv'},
 'encoding': {'x': {'field': 'Genre', 'type': 'nominal'},
              'y': {'aggregate': 'mean',
                    'axis': {'format': 's'},
                    'field': 'Worldwide Gross',
                    'type': 'quantitative'}},
 'mark': {'tooltip': True, 'type': 'bar'},
 'transform': [{'filter': 'lower(datum["Worldwide Gross"]) > 100000000.0'},
               {'filter': {'field': 'Creative Type',
                           'oneOf': ['Science Fiction', 'Fantasy']}}]}
show_transform_process(vl_spec, data_position, 'Title')

------Overview of all the relevant fields-------


,Worldwide Gross,Title,Genre,Creative Type
0,25728961,From Dusk Till Dawn,Horror,Fantasy
1,148345997,Broken Arrow,Action,Contemporary Fiction
2,20278055,City Hall,Drama,Contemporary Fiction
3,38623460,Happy Gilmore,Comedy,Contemporary Fiction
4,51204567,Fargo,Thriller,Contemporary Fiction
...,...,...,...,...
704,493296,The City of Your Final Destination,Drama,Contemporary Fiction
705,310885538,Robin Hood,Action,Historical Fiction
706,753830280,Inception,Thriller,Science Fiction
707,43149143,Scott Pilgrim vs. The World,Comedy,Contemporary Fiction


------After filter No.1-------
{'type': 'filter_str', 'field': 'Worldwide Gross', 'symbol': '>', 'value': 100000000.0}


,Worldwide Gross,Title,Genre,Creative Type
1,148345997,Broken Arrow,Action,Contemporary Fiction
6,495900000,Twister,Action,Contemporary Fiction
7,104364680,Dragonheart,Adventure,Fantasy
9,336069511,The Rock,Action,Contemporary Fiction
10,102825796,The Cable Guy,Comedy,Contemporary Fiction
...,...,...,...,...
695,172450423,Couples Retreat,Comedy,Contemporary Fiction
697,113190972,Law Abiding Citizen,Thriller,Contemporary Fiction
701,766812167,2012,Action,Science Fiction
705,310885538,Robin Hood,Action,Historical Fiction


------After filter No.2-------
{'type': 'filter_dict', 'field': 'Creative Type', 'range': ['Science Fiction', 'Fantasy']}


,Worldwide Gross,Title,Genre,Creative Type
15,817400878,Independence Day,Adventure,Science Fiction
43,101371017,Mars Attacks!,Comedy,Science Fiction
55,263900000,The Fifth Element,Action,Science Fiction
60,587790539,Men in Black,Adventure,Science Fiction
61,165900000,Contact,Drama,Science Fiction
...,...,...,...,...
640,397912118,The Mummy: Tomb of the Dragon Emperor,Adventure,Fantasy
662,396439854,Twilight,Drama,Fantasy
668,329809326,The Curious Case of Benjamin Button,Drama,Fantasy
674,381687380,Monsters vs. Aliens,Adventure,Fantasy


------After Aggregation2-------
{'type': 'aggregation', 'op': 'mean', 'field': 'Worldwide Gross', 'groupby': 'Genre'}


,Genre,Title,Worldwide Gross,mean Worldwide Gross
0,Action,2012,766812167,372087111.0
1,Action,End of Days,212026975,372087111.0
2,Action,Godzilla,376000000,372087111.0
3,Action,Minority Report,358824714,372087111.0
4,Action,Resident Evil: Extinction,146162920,372087111.0
...,...,...,...,...
61,Romantic Comedy,Enchanted,340384141,340384141.0
62,Thriller,District 9,206552113,383012970.5
63,Thriller,Inception,753830280,383012970.5
64,Thriller,Signs,408265690,383012970.5


In [6]:
my_df = My_DataFrame(pd.read_csv('debug_tb.csv'))
my_df['CN']=my_df.groupby(['Genre'],as_index=False).rank(method='first').iloc[:,0] 
my_df['RN']=my_df.groupby(['Genre'],as_index=False).rank(method='max').iloc[:,0] 
my_df
# my_df.loc[0,"Title"]

,Genre,Title,Worldwide Gross,mean Worldwide Gross,CN,RN
0,Action,2012,766812167,3.720871e+08,16.0,16.0
1,Action,End of Days,212026975,3.720871e+08,7.0,7.0
2,Action,Godzilla,376000000,3.720871e+08,11.0,11.0
3,Action,Minority Report,358824714,3.720871e+08,10.0,10.0
4,Action,Resident Evil: Extinction,146162920,3.720871e+08,4.0,4.0
5,Action,Sin City,158753820,3.720871e+08,5.0,5.0
6,Action,Starship Troopers,121100000,3.720871e+08,2.0,2.0
7,Action,Surrogates,119668350,3.720871e+08,1.0,1.0
8,Action,The Fifth Element,263900000,3.720871e+08,8.0,8.0
9,Action,The Forbidden Kingdom,129075270,3.720871e+08,3.0,3.0
